In [2]:
!pip install langchain
!pip install pypdf
!pip install chromadb
!pip install pysqlite3
!pip install pysqlite3-binary

In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [4]:
!pip install langchain_huggingface

In [5]:
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [6]:
text_loader = TextLoader("/kaggle/input/resume/state_of_union.txt")
text_document = text_loader.load()

In [10]:
print (text_document[0])

page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

Groups of citizens blocking tanks with their

In [11]:
pdf_loader = PyPDFLoader("/kaggle/input/resume/michael_resume.pdf")
pdf_pages = pdf_loader.load_and_split()
# display first page content from pdf document.
print(pdf_pages[0])

page_content='CURRICULUM VITAE :  
M ichael M . Scott OBE, B.Sc., Dip.Ed  
 
Home address:  Strome House     Date of Birth: 10.5.51 
   North Strome     Place of Birth: Edinburgh 
   Lochcarron     Married to Sue Scott; 2 stepchildren 
   Ross-shire, IV54 8YJ 
Telephone (work): 01520 722901     Website: www.mmscott.co.uk 
Telephone (home):  01520 722588    E-mail:  MSStrome@aol.com 
 
Awarded OBE in Queen’s Birthday Honours, June 2005, “for services to biodiversity conservation in 
Scotland”. 
Awarded Planta Europa ‘Silver Lead’ Award in September 2007, “for excellent work in European wild plant 
conservation”. 
 
Education 
Primary education: George Heriots School, Edinburgh (1956-1962) 
Secondary education:  Madras College, St Andrews (1962-69). 
Further education: University of Aberdeen (1969-1974): 
    Bachelor of Science (Honours; upper second) in Botany, 1973 
    Diploma of Education, 1974 
   Aberdeen College of Education (1973 - 1974): 
    Certificate of Education in seconda

In [12]:
pdf_loader = PyPDFLoader("/kaggle/input/resume/Data_Engineer_Resume.pdf")
pdf_pages1 = pdf_loader.load_and_split()

In [13]:
document = pdf_pages + pdf_pages1

In [14]:
text_splitted = RecursiveCharacterTextSplitter(chunk_size = 200, chunk_overlap = 30)
tokens = text_splitted.split_documents(document)

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings
model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf_embedding = HuggingFaceEmbeddings(
                                    model_name = model_name,
                                    model_kwargs=model_kwargs,
                                    encode_kwargs=encode_kwargs
                                )

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [22]:
text = tokens[0].page_content

In [23]:
# Embedding for single string
hf_embed_result = hf_embedding.embed_documents(tokens[0].page_content)

In [24]:
len(hf_embed_result)

164

In [25]:
api_key = ""

In [26]:
openai_embeddings = OpenAIEmbeddings(openai_api_key = api_key)
# Embedding for single string
openai_embed_result = openai_embeddings.embed_documents(text)

In [27]:
len(openai_embed_result)

164

In [28]:
chroma_db = Chroma.from_documents(tokens, hf_embedding, persist_directory = "/kaggle/working/outputdb")

In [29]:
chroma_db.persist()

<ipython-input-29-733b9d63d7ec>:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_db.persist()


In [30]:
search_result = chroma_db.similarity_search_with_score("What is the candidate skill set?", k = 1)

In [32]:
search_result

[(Document(metadata={'page': 0, 'page_label': '1', 'source': '/kaggle/input/resume/Data_Engineer_Resume.pdf'}, page_content='pipelines, ETL processes, and data warehouses. Proficient in SQL, Python, and big data\ntechnologies like Hadoop and Spark. Strong analytical skills with a focus on optimizing data'),
  1.0486767292022705)]

In [33]:
search_result[0][0].page_content

'pipelines, ETL processes, and data warehouses. Proficient in SQL, Python, and big data\ntechnologies like Hadoop and Spark. Strong analytical skills with a focus on optimizing data'